In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

TODO: Add correct link here

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/googleapis/genai-toolbox/tree/main/docs/en/getting-started/colab_quickstart.ipynb)

# Quickstart

This guide demonstrates how to quickly run Toolbox end-to-end in Google Colab using Python, PostgreSQL, and LlamaIndex.

Within this Colab environment, you'll
- Set up a `PostgreSQL database`
- Launch a Toolbox server,
- Connect to Toolbox and develop a sample `Hotel Booking` application.

## Step 1: Set up your database

In this section, we will
1. Create a database
1. Create a user to access the database
1. Insert dummy data into the database.

In [ ]:
# Install postgresql to run a DB server on colab
%%shell

sudo apt-get -y -qq update > /dev/null 2>&1
sudo apt-get -y -qq install postgresql > /dev/null 2>&1

In [ ]:
# Start the postgresql server.
!sudo service postgresql start

 * Starting PostgreSQL 14 database server
   ...done.


In [ ]:
# Check that postgres is running
!sudo lsof -i :5432

COMMAND   PID     USER   FD   TYPE DEVICE SIZE/OFF NODE NAME
postgres 7582 postgres    5u  IPv4 159385      0t0  TCP localhost:postgresql (LISTEN)


In [ ]:
# Create a dedicated database and a user to access our DB securely
%%shell

sudo -u postgres psql << EOF
CREATE USER toolbox_user WITH PASSWORD 'my-password';
CREATE DATABASE toolbox_db;
GRANT ALL PRIVILEGES ON DATABASE toolbox_db TO toolbox_user;
ALTER DATABASE toolbox_db OWNER TO toolbox_user;
EOF

CREATE ROLE
CREATE DATABASE
GRANT
ALTER DATABASE


> **Tip:** For a real application, it’s best to follow the principle of least permission and only grant the privileges your application needs.



In [ ]:
# Connect to the database with the new user and create a hotels table.
%%shell

export PGPASSWORD=my-password
psql -h 127.0.0.1 -U toolbox_user -d toolbox_db --no-password << EOF
CREATE TABLE hotels(
   id            INTEGER NOT NULL PRIMARY KEY,
   name          VARCHAR NOT NULL,
   location      VARCHAR NOT NULL,
   price_tier    VARCHAR NOT NULL,
   checkin_date  DATE    NOT NULL,
   checkout_date DATE    NOT NULL,
   booked        BIT     NOT NULL
);
INSERT INTO hotels(id, name, location, price_tier, checkin_date, checkout_date, booked)
VALUES
  (1, 'Hilton Basel', 'Basel', 'Luxury', '2024-04-22', '2024-04-20', B'0'),
  (2, 'Marriott Zurich', 'Zurich', 'Upscale', '2024-04-14', '2024-04-21', B'0'),
  (3, 'Hyatt Regency Basel', 'Basel', 'Upper Upscale', '2024-04-02', '2024-04-20', B'0'),
  (4, 'Radisson Blu Lucerne', 'Lucerne', 'Midscale', '2024-04-24', '2024-04-05', B'0'),
  (5, 'Best Western Bern', 'Bern', 'Upper Midscale', '2024-04-23', '2024-04-01', B'0'),
  (6, 'InterContinental Geneva', 'Geneva', 'Luxury', '2024-04-23', '2024-04-28', B'0'),
  (7, 'Sheraton Zurich', 'Zurich', 'Upper Upscale', '2024-04-27', '2024-04-02', B'0'),
  (8, 'Holiday Inn Basel', 'Basel', 'Upper Midscale', '2024-04-24', '2024-04-09', B'0'),
  (9, 'Courtyard Zurich', 'Zurich', 'Upscale', '2024-04-03', '2024-04-13', B'0'),
  (10, 'Comfort Inn Bern', 'Bern', 'Midscale', '2024-04-04', '2024-04-16', B'0');
SELECT * from hotels;
EOF

CREATE TABLE
INSERT 0 10
 id |          name           | location |   price_tier   | checkin_date | checkout_date | booked 
----+-------------------------+----------+----------------+--------------+---------------+--------
  1 | Hilton Basel            | Basel    | Luxury         | 2024-04-22   | 2024-04-20    | 0
  2 | Marriott Zurich         | Zurich   | Upscale        | 2024-04-14   | 2024-04-21    | 0
  3 | Hyatt Regency Basel     | Basel    | Upper Upscale  | 2024-04-02   | 2024-04-20    | 0
  4 | Radisson Blu Lucerne    | Lucerne  | Midscale       | 2024-04-24   | 2024-04-05    | 0
  5 | Best Western Bern       | Bern     | Upper Midscale | 2024-04-23   | 2024-04-01    | 0
  6 | InterContinental Geneva | Geneva   | Luxury         | 2024-04-23   | 2024-04-28    | 0
  7 | Sheraton Zurich         | Zurich   | Upper Upscale  | 2024-04-27   | 2024-04-02    | 0
  8 | Holiday Inn Basel       | Basel    | Upper Midscale | 2024-04-24   | 2024-04-09    | 0
  9 | Courtyard Zurich        | Z

In [ ]:
# Check that database is running
!sudo lsof -i :5432

COMMAND   PID     USER   FD   TYPE DEVICE SIZE/OFF NODE NAME
postgres 7582 postgres    5u  IPv4 159385      0t0  TCP localhost:postgresql (LISTEN)


## Step 2: Install and configure toolbox

In this section, we will
1. Download the latest version of the toolbox binary.
2. Create a toolbox config file.
3. Start a toolbox server using the config file.



Download the [latest](https://github.com/googleapis/genai-toolbox/releases) version of Toolbox as a binary.

In [ ]:
! curl -O https://storage.googleapis.com/genai-toolbox/v0.2.0/linux/amd64/toolbox

# Make the binary executable
! chmod +x toolbox

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 61.2M  100 61.2M    0     0  37.7M      0  0:00:01  0:00:01 --:--:-- 37.7M


In [ ]:
TOOLBOX_BINARY_PATH = "/content/toolbox"
SERVER_PORT = 5000

> Note: To include a literal dollar sign (e.g., $1) as part of your SQL statement within the Python string for tools.yml, you must escape both the backslash and the dollar sign. Use \\\$1 in Python to output \$1 in the tools.yml file.

> Note: You can also set up Colab secrets to store any sensitive information like passwords. You can easily add secrets through the left panel:

<img src="https://services.google.com/fh/files/misc/colab_secret.png" alt="Colab Secrets" width="400"/>


Create a tools file with the following functions:

- `Database Connection`: It includes details for connecting to our hotels database.
- `Tool Definitions`: It defines five tools for database interaction:
  - `search-hotels-by-name`
  - `search-hotels-by-location`
  - `book-hotel`
  - `update-hotel`
  - `cancel-hotel`

Our application will leverage these tools to interact with the hotels database.

For detailed configuration options, please refer to the [toolbox documentation](https://googleapis.github.io/genai-toolbox/getting-started/configure/).



In [ ]:
# Create a tools file at runtime.
# You can also upload a tools file and use that to run toolbox.
tools_file_name = "tools.yml"
file_content = f"""
sources:
  my-pg-source:
    kind: postgres
    host: 127.0.0.1
    port: 5432
    database: toolbox_db
    user: toolbox_user
    password: my-password
tools:
  search-hotels-by-name:
    kind: postgres-sql
    source: my-pg-source
    description: Search for hotels based on name.
    parameters:
      - name: name
        type: string
        description: The name of the hotel.
    statement: SELECT * FROM hotels WHERE name ILIKE '%' || \$1 || '%';
  search-hotels-by-location:
    kind: postgres-sql
    source: my-pg-source
    description: Search for hotels based on location.
    parameters:
      - name: location
        type: string
        description: The location of the hotel.
    statement: SELECT * FROM hotels WHERE location ILIKE '%' || \$1 || '%';
  book-hotel:
    kind: postgres-sql
    source: my-pg-source
    description: >-
       Book a hotel by its ID. If the hotel is successfully booked, returns a NULL, raises an error if not.
    parameters:
      - name: hotel_id
        type: string
        description: The ID of the hotel to book.
    statement: UPDATE hotels SET booked = B'1' WHERE id = \$1;
  update-hotel:
    kind: postgres-sql
    source: my-pg-source
    description: >-
      Update a hotel's check-in and check-out dates by its ID. Returns a message
      indicating  whether the hotel was successfully updated or not.
    parameters:
      - name: hotel_id
        type: string
        description: The ID of the hotel to update.
      - name: checkin_date
        type: string
        description: The new check-in date of the hotel.
      - name: checkout_date
        type: string
        description: The new check-out date of the hotel.
    statement: >-
      UPDATE hotels SET checkin_date = CAST(\$2 as date), checkout_date = CAST(\$3
      as date) WHERE id = \$1;
  cancel-hotel:
    kind: postgres-sql
    source: my-pg-source
    description: Cancel a hotel by its ID.
    parameters:
      - name: hotel_id
        type: string
        description: The ID of the hotel to cancel.
    statement: UPDATE hotels SET booked = B'0' WHERE id = \$1;
"""

In [ ]:
# Write the file content into the tools file.
! echo "{file_content}" > "{tools_file_name}"

In [ ]:
TOOLS_FILE_PATH = f"/content/{tools_file_name}"

In [ ]:
# Start a toolbox server
! nohup {TOOLBOX_BINARY_PATH} --tools_file {TOOLS_FILE_PATH} -p {SERVER_PORT} > toolbox.log 2>&1 &

In [ ]:
# Check if toolbox is running
!sudo lsof -i :{SERVER_PORT}

COMMAND  PID USER   FD   TYPE DEVICE SIZE/OFF NODE NAME
toolbox 7640 root    6u  IPv4 161323      0t0  TCP localhost:5000 (LISTEN)


## Step 3: Connect your agent to Toolbox

In this section, you will
1. Establish a connection to the tools by creating a Toolbox client.
2. Build an agent that leverages the tools and an LLM for `Hotel Booking` functionality.


> You need to authenticate as an IAM user so this notebook can access your Google Cloud Project. This access is necessary to use Google's LLM models.

In [ ]:
# Run this and allow access through the pop-up
from google.colab import auth

auth.authenticate_user()

In [ ]:
# @markdown Please fill in the value below with your GCP project ID and then run the cell.

# Please fill in these values.
project_id = "twisha-dev"  # @param {type:"string"}

# Quick input validations.
assert project_id, "⚠️ Please provide a Google Cloud project ID"

# Configure gcloud.
!gcloud config set project {project_id}

Updated property [core/project].


> You can either use LangGraph or LlamaIndex to develop a toolbox based application. Run one of the `Connect Using LangGraph` or `Connect using LlamaIndex` sections below.



### Connect Using LangGraph

In [ ]:
# Install the Toolbox Langchain package
!pip install toolbox-langchain --quiet
!pip install langgraph --quiet

# Install the Langchain llm package
# TODO(developer): replace this with another model if needed
! pip install langchain-google-vertexai --quiet
# ! pip install langchain-google-genai
# ! pip install langchain-anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.9 MB/s eta 0:00:00


Create a LangGraph Hotel Agent which can Search, Book and Cancel hotels.

In [ ]:
from langgraph.prebuilt import create_react_agent
# TODO(developer): replace this with another import if needed
from langchain_google_vertexai import ChatVertexAI
# from langchain_google_genai import ChatGoogleGenerativeAI
# from langchain_anthropic import ChatAnthropic
from langgraph.checkpoint.memory import MemorySaver

from toolbox_langchain import ToolboxClient

prompt = """
  You're a helpful hotel assistant. You handle hotel searching, booking and
  cancellations. When the user searches for a hotel, mention it's name, id,
  location and price tier. Always mention hotel ids while performing any
  searches. This is very important for any operations. For any bookings or
  cancellations, please provide the appropriate confirmation. Be sure to
  update checkin or checkout dates if mentioned by the user.
  Don't ask for confirmations from the user.
"""

queries = [
    "Find hotels in Basel with Basel in it's name.",
    "Can you book the Hilton Basel for me?",
    "Oh wait, this is too expensive. Please cancel it and book the Hyatt Regency instead.",
    "My check in dates would be from April 10, 2024 to April 19, 2024.",
]

# Create an LLM to bind with the agent.
# TODO(developer): replace this with another model if needed
model = ChatVertexAI(model_name="gemini-1.5-pro", project=project_id)
# model = ChatGoogleGenerativeAI(model="gemini-1.5-pro")
# model = ChatAnthropic(model="claude-3-5-sonnet-20240620")

# Load the tools from the Toolbox server
client = ToolboxClient("http://127.0.0.1:5000")
tools = client.load_toolset()

# Create a Langraph agent
agent = create_react_agent(model, tools, checkpointer=MemorySaver())
config = {"configurable": {"thread_id": "thread-1"}}
for query in queries:
    inputs = {"messages": [("user", prompt + query)]}
    response = agent.invoke(inputs, stream_mode="values", config=config)
    print(response["messages"][-1].content)

I found 3 hotels in Basel: The Hilton Basel (id: 1) is a Luxury hotel, Hyatt Regency Basel (id: 3) is an Upper Upscale hotel and Holiday Inn Basel (id: 8) is an Upper Midscale hotel. Do you have any other requests? 

You are booked at the Hilton Basel. Have a great stay! 

Your booking at Hilton Basel has been cancelled and you are now booked at Hyatt Regency Basel. 

Your booking at Hyatt Regency Basel has been updated! Your new check-in date is April 10, 2024 and your new check-out date is April 19, 2024. 



### Connect using LlamaIndex

In [ ]:
# Install the Toolbox LlamaIndex package
!pip install toolbox-llamaindex --quiet

# Install the llamaindex llm package
# TODO(developer): replace this with another model if needed
! pip install llama-index-llms-google-genai --quiet
# ! pip install llama-index-llms-anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.5/264.5 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 39.3 MB/s eta 0:00:00


Create a LlamaIndex Hotel Agent which can Search, Book and Cancel hotels.

In [ ]:
import asyncio
import os

from llama_index.core.agent.workflow import AgentWorkflow

from llama_index.core.workflow import Context

# TODO(developer): replace this with another import if needed
from llama_index.llms.google_genai import GoogleGenAI
# from llama_index.llms.anthropic import Anthropic

from toolbox_llamaindex import ToolboxClient

prompt = """
  You're a helpful hotel assistant. You handle hotel searching, booking and
  cancellations. When the user searches for a hotel, mention it's name, id,
  location and price tier. Always mention hotel ids while performing any
  searches. This is very important for any operations. For any bookings or
  cancellations, please provide the appropriate confirmation. Be sure to
  update checkin or checkout dates if mentioned by the user.
  Don't ask for confirmations from the user.
"""

queries = [
    "Find hotels in Basel with Basel in it's name.",
    "Can you book the Hilton Basel for me?",
    "Oh wait, this is too expensive. Please cancel it and book the Hyatt Regency instead.",
    "My check in dates would be from April 10, 2024 to April 19, 2024.",
]

async def run_agent():
    # Create an LLM to bind with the agent.
    # TODO(developer): replace this with another model if needed
    llm = GoogleGenAI(
        model="gemini-1.5-pro",
        vertexai_config={"project": project_id, "location": "us-central1"},
    )
    # llm = GoogleGenAI(
    #     api_key=os.getenv("GOOGLE_API_KEY"),
    #     model="gemini-1.5-pro",
    # )
    # llm = Anthropic(
    #   model="claude-3-7-sonnet-latest",
    #   api_key=os.getenv("ANTHROPIC_API_KEY")
    # )

    # Load the tools from the Toolbox server
    client = ToolboxClient("http://127.0.0.1:5000")
    tools = client.load_toolset()

    # Create a LlamaIndex agent
    agent = AgentWorkflow.from_tools_or_functions(
        tools,
        llm=llm,
        system_prompt=prompt,
    )

    # Run the agent
    ctx = Context(agent)
    for query in queries:
        response = await agent.run(user_msg=query, ctx=ctx)
        print(f"---- {query} ----")
        print(str(response))

await run_agent()

---- Find hotels in Basel with Basel in it's name. ----
I found 3 hotels in Basel: 
Hotel id 1 - Hilton Basel, located in Basel. It's a Luxury hotel.
Hotel id 3 - Hyatt Regency Basel, located in Basel. It's a Upper Upscale hotel.
Hotel id 8 - Holiday Inn Basel, located in Basel. It's a Upper Midscale hotel. 

---- Can you book the Hilton Basel for me? ----
You are all set. The Hilton Basel is booked for you. 

---- Oh wait, this is too expensive. Please cancel it and book the Hyatt Regency instead. ----
The Hilton Basel is cancelled and Hyatt Regency Basel is booked for you. 

---- My check in dates would be from April 10, 2024 to April 19, 2024. ----
Your booking for Hyatt Regency Basel has been updated with the new check-in and check-out dates. 



### Observe the output

You can see that the `Hyatt Regency Basel` has been booked for the correct dates.

In [ ]:
%%shell

export PGPASSWORD=my-password
psql -h 127.0.0.1 -U toolbox_user -d toolbox_db --no-password << EOF
SELECT * from hotels;
EOF

 id |          name           | location |   price_tier   | checkin_date | checkout_date | booked 
----+-------------------------+----------+----------------+--------------+---------------+--------
  2 | Marriott Zurich         | Zurich   | Upscale        | 2024-04-14   | 2024-04-21    | 0
  4 | Radisson Blu Lucerne    | Lucerne  | Midscale       | 2024-04-24   | 2024-04-05    | 0
  5 | Best Western Bern       | Bern     | Upper Midscale | 2024-04-23   | 2024-04-01    | 0
  6 | InterContinental Geneva | Geneva   | Luxury         | 2024-04-23   | 2024-04-28    | 0
  7 | Sheraton Zurich         | Zurich   | Upper Upscale  | 2024-04-27   | 2024-04-02    | 0
  8 | Holiday Inn Basel       | Basel    | Upper Midscale | 2024-04-24   | 2024-04-09    | 0
  9 | Courtyard Zurich        | Zurich   | Upscale        | 2024-04-03   | 2024-04-13    | 0
 10 | Comfort Inn Bern        | Bern     | Midscale       | 2024-04-04   | 2024-04-16    | 0
  1 | Hilton Basel            | Basel    | Luxury         

## Optional: Cleanup

Executing this will terminate the processes running on the database and Toolbox ports.

This is necessary before re-running the startup cells for these services to prevent `port already in use` errors.

In [ ]:
import subprocess
import os
import signal

def kill_processes_on_port(port):
    """Kills all processes listening on a given port using lsof and kill.

    Args:
        port: The port number (as an integer).
    """
    try:
        result = subprocess.run(['lsof', '-t', '-i', f':{port}'], capture_output=True, text=True, check=True)
        pids_str = result.stdout.strip()

        if not pids_str:
            print(f"No processes found listening on port {port}.")
            return

        pids = [int(pid) for pid in pids_str.split('\n')]
        print(f"Found processes on port {port}: {pids}")

        for pid in pids:
            try:
                os.kill(pid, signal.SIGKILL)
                print(f"Process {pid} killed.")
            except OSError as e:
                print(f"Error killing process {pid}: {e}")

    except subprocess.CalledProcessError as e:
        print(f"Error running lsof: {e}")
        print(f"lsof output: {e.output}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

In [ ]:
kill_processes_on_port(5432)

Found processes on port 5432: [3572, 3622, 3629]
Process 3572 killed.
Process 3622 killed.
Process 3629 killed.


In [ ]:
# Verify that the database process is killed
!sudo lsof -i :5432